In [21]:
import cv2
import time
import numpy as np

In [22]:
!pip install mediapipe

In [23]:
import importlib
import Hand_Tracker as ht
importlib.reload(ht)

<module 'Hand_Tracker' from 'C:\\Users\\Lenovo\\Hand_Tracker.py'>

In [24]:
!pip install pycaw

In [25]:
cap = cv2.VideoCapture(0)        # default cam
default_width = cap.get(3)
default_height = cap.get(4)
print(default_width, 'x' , default_height)  # checking our camera's default resolution


640.0 x 480.0


In [33]:
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

cap = cv2.VideoCapture(0)       
cap.set(3, 640)
cap.set(4, 480)
pTime = 0

detector = ht.handDetector(detectionCon=0.75, maxHands=1)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0
volumeLocked = False

try:
    while True:
        success, img = cap.read()
        if not success:
            print("Failed to capture image")
            break
        img = detector.findHands(img)
        lmList, bbox = detector.findPosition(img, draw=False)
        if len(lmList) != 0:
            fingers = detector.fingersUp()

            if fingers[3] == 0:
                volumeLocked = True
            
            elif fingers == [1, 1, 1, 1, 1]:
                volumeLocked = False
        
            if not volumeLocked:
                x1, y1 = lmList[4][1], lmList[4][2]
                x2, y2 = lmList[8][1], lmList[8][2]
        
                cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        
                length = math.hypot(x2 - x1, y2 - y1)
        
                minLength = 30
                maxLength = 250
        
                volPer = np.interp(length, [minLength, maxLength], [0, 100])
                volBar = np.interp(length, [minLength, maxLength], [400, 150])
                volScalar = volPer / 100
                
                smoothVol = 0.8 * volScalar + 0.2 * volume.GetMasterVolumeLevelScalar()
                volume.SetMasterVolumeLevelScalar(smoothVol, None)
        
                if abs(length - maxLength) < 10 or abs(length - minLength) < 10:
                    volPer = round(volPer)



        cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 2)
        cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, f'{int(volPer)} %', (40, 430), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0), 3)
        if volumeLocked:
            cv2.putText(img, 'VOLUME LOCKED', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 3)

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except KeyboardInterrupt:
    print("Keyboard interrupt detected. Releasing camera...")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Camera released and windows closed.")


Keyboard interrupt detected. Releasing camera...
Camera released and windows closed.
